In [15]:
import sys
print(sys.executable)

c:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\credit-card-fraud-detection-model-FZHIqfLr-py3.13\Scripts\python.exe


In [16]:
import pandas as pd

Intro:


# Data Exploration – Dataset Tables

In this section we will first try to explore our raw dataset files to understand 
its structure, columns, datatypes, and potential issues (missing values, duplicates, etc.), before we preform any merge.

<br><u> We have 2 tables: </u></br>
* **customers.csv**
* **credit_card_fraud.csv**

### <u> Data Exploration – Customers Dataset : </u>

In [ ]:
transactions_df = pd.read_csv("C:/Users/User/Documents/GitHub/Credit_Card_Fraud_Detection_Model/data/credit_card_fraud.csv", index_col=0)
customers_df = pd.read_csv("C:/Users/User/Documents/GitHub/Credit_Card_Fraud_Detection_Model/data/customers.csv", sep="|")



In [30]:
def compare_columns(df1, name1, df2, name2):
    """
    Compares columns between two DataFrames.
    Prints:
    1. Common columns and their data types in both datasets.
    2. Columns unique to each dataset.
    """
    set1, set2 = set(df1.columns), set(df2.columns)
    common = set1.intersection(set2)
    only_in_1 = set1 - set2
    only_in_2 = set2 - set1
    
    print("="*60)
    print("🔹 Common Columns with Data Types")
    if common:
        comparison = {
            "Column": [],
            f"{name1} dtype": [],
            f"{name2} dtype": []
        }
        for col in sorted(common):
            comparison["Column"].append(col)
            comparison[f"{name1} dtype"].append(df1[col].dtype)
            comparison[f"{name2} dtype"].append(df2[col].dtype)
        result = pd.DataFrame(comparison)
        print(result.to_string(index=False))
    else:
        print("No common columns found.")
    
    print("\n" + "="*60)
    print(f"🔹 Columns only in {name1}:")
    if only_in_1:
        print(sorted(list(only_in_1)))
    else:
        print("No unique columns found.")
    
    print("\n" + "="*60)
    print(f"🔹 Columns only in {name2}:")
    if only_in_2:
        print(sorted(list(only_in_2)))
    else:
        print("No unique columns found.")
    print("="*60)



In [31]:
compare_columns(transactions_df, "Transactions", customers_df, "Customers")


🔹 Common Columns with Data Types
  Column Transactions dtype Customers dtype
acct_num              int64           int64
  cc_num              int64           int64
    city             object          object
city_pop              int64           int64
     dob             object          object
   first             object          object
  gender             object          object
     job             object          object
    last             object          object
     lat            float64         float64
    long            float64         float64
 profile             object          object
     ssn             object          object
   state             object          object
  street             object          object
     zip              int64           int64

🔹 Columns only in Transactions:
['Unnamed: 0', 'amt', 'category', 'is_fraud', 'merch_lat', 'merch_long', 'merchant', 'trans_date', 'trans_num', 'trans_time', 'unix_time']

🔹 Columns only in Customers:
No unique columns

In [32]:
transactions_df.head(5)

,Unnamed: 0,ssn,cc_num,first,last,gender,street,city,state,zip,...,trans_num,trans_date,trans_time,unix_time,category,amt,is_fraud,merchant,merch_lat,merch_long
0,0,750-09-7342,6011237648640631,Amanda,Morris,F,144 Bowers Route,Tracy,CA,95376,...,52226c099d060def61a7e0d3de214e4b,2020-02-15,02:48:17,1581715097,misc_net,4.34,1,"fraud_Moore, Dibbert and Koepp",38.724545,-120.916415
1,1,750-09-7342,6011237648640631,Amanda,Morris,F,144 Bowers Route,Tracy,CA,95376,...,6495d95c66d7d504d7352f2b8c105d3c,2020-02-15,01:29:36,1581710376,misc_net,11.20,1,fraud_Dach-Nader,37.288981,-121.791754
2,2,750-09-7342,6011237648640631,Amanda,Morris,F,144 Bowers Route,Tracy,CA,95376,...,bc0bfcd9e0132b171e538192b0e27777,2020-02-15,11:40:04,1581747004,grocery_pos,282.55,1,fraud_Pacocha-O'Reilly,38.461530,-120.624583
3,3,750-09-7342,6011237648640631,Amanda,Morris,F,144 Bowers Route,Tracy,CA,95376,...,483c446c7b56a11f0d4538056eb79984,2020-02-15,01:20:48,1581709848,gas_transport,294.94,1,"fraud_Kutch, Hermiston and Farrell",37.920539,-121.439390
4,4,750-09-7342,6011237648640631,Amanda,Morris,F,144 Bowers Route,Tracy,CA,95376,...,fe2209222832f626cd2cfdcb6d7130b5,2020-02-15,01:15:19,1581709519,misc_net,326.40,1,fraud_Kuphal-Bartoletti,38.123850,-120.843755


### <u> Data Exploration – credit_card_fraud Dataset </u>
